In [ ]:
"""Sofisticated composition of decorators"""
from functools import wraps, reduce
from decorator import decorator

import nographs as nog
from graphviz import Digraph

import torch
# import torch.nn as nn
# import torch.nn.functional as F
from torchvision import models

In [ ]:
nn_model = models.convnext_tiny()
#nn_model

In [ ]:
node_style ={"style": 'filled',            "shape": 'box',
             "align": 'left',              "fontsize": '12',
             "ranksep":'0.01',             "height":'0.02',   "width":'0.04',}
gr_style = {"linelength": '16', "rankdir": 'LR'}#"landscape":'True',

def adapter_graphviz( wrapped ):
    """translation to GraphViz visitor"""
    dot = Digraph( node_attr = node_style, graph_attr = gr_style, )
    @wraps( wrapped )
    def wrapper_bg( _edge, _trav ):
        dot.node( str(_edge), str(_edge) )
        for result in wrapped(_edge, _trav):
            edge = dot.edge( str(_edge), str(result[0]), str(result[1]), )
            yield result
    wrapper_bg.dot = dot
    return wrapper_bg

In [ ]:
def chain_two( f, g ):
    """chain two functions"""
    def lll(x):
        res = f(g(x))
        print( f"*{x}*\n*{f}*\n*{g}*\n*{res}*" )
        return res
    return lll#lambda x:f(g(x))

In [ ]:
def chain_functions( *funcs ):
    """chain function calling"""
    return reduce( chain_two, funcs)

In [ ]:
def while_changed( *wrappers ):
    """repeat transformations, until no opportunities"""
    cycle = chain_functions( *wrappers )
    def _loop_static_flow( body ):
        def _runtime_flow( _item, _trav ):
            changed = True
            while changed:
                changed = False
                ( _item, _trav ) = cycle( body )( _item, _trav )
    return _loop_static_flow

In [ ]:
@decorator
def filter_boring( wrapped, _edge, _trav ):
    """Decorator for filtering, lets shrink graph a bit"""
    stop_words = {'__dir__', '__str__','__repr__','__doc__','__hash__'}
    yield from filter( lambda e:e[1] not in stop_words, wrapped( _edge, _trav ) )

In [ ]:
@decorator
def expand_iterables( wrapped, item, _trav ):
    """Decorator for sequenses substitution"""
    # print(f"{hasattr( item, '__iter__' )} {item=}")
    if hasattr( item, '__iter__' ):
        for sub in item:
            yield from expand_iterables( wrapped, sub, _trav )
    else:
        yield from wrapped( item, _trav )

In [ ]:
@adapter_graphviz
#@expand_iterables
@while_changed(filter_boring,expand_iterables,)
#@filter_boring
def all_attr(var, _):
    for attr_name, _ in zip(dir(var), range(3)):
        print(attr_name)
        try:
            attr = getattr(var, attr_name)
            if hasattr( attr, "__call__"):
                # try:
                    attr = attr()
                # except Exception as ex:
                #     attr = str(ex)[:50]
            if attr is not None: #hasattr( attr, "__hash__"):
                print((str(attr), str(attr_name)), type(attr))
                yield (str(attr), str(attr_name))
        except:
            print("$$$trouble")
            continue

trav_forward = nog.TraversalBreadthFirst(next_labeled_edges=all_attr)
trav_forward.start_from( start_vertices=list(nn_model.children()), build_paths=True )
#trav_forward.start_from( nn_model.children(), build_paths=True )
for _, _ in zip(trav_forward, range(3)):
    pass

In [ ]:
#print(all_attr.dot.source)              # pylint: disable=pointless-statement
#all_attr.dot

In [ ]:
dir(nn_model)
type(nn_model)
type(nn_model).__bases__

In [ ]:
#list(nn_model.children())

In [ ]:
issubclass(type(nn_model), torch.nn.modules.module.Module)

In [ ]:
#dir(nn_model)

In [ ]:
def plusone(x):
    return x+1

In [ ]:
chain_functions(plusone,plusone,plusone,)(10)